<a href="https://colab.research.google.com/github/sepehrilami/DataScienceInternship-AIMed/blob/master/PLS/PLS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#  Importing!

import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torch.autograd import Variable

from __future__ import print_function
#%matplotlib inline
import argparse
import os
import random
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML
from PIL import Image
from torch.utils.data import Dataset, DataLoader


In [ ]:
# custom weights initialization
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.01)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.01)
        nn.init.constant_(m.bias.data, 0)

Defining Depthwise Separable (DS) Convolution And Dilated Residual Dense Block (DRDB)

In [ ]:
class DSconv(nn.Module):
  def __init__(in_channels, out_channels, DSkernel, DSstride, DSpadding, dilation = True):
    super(DSconv, self).__init__()

    self.dsconv = nn.Sequential(

        #nn.Conv3d(in_chans, in_chans * k, kernel_size, groups = in_chans)
        #here k is M * N or just M or 1 ??
        nn.Conv3d(in_channels, in_channels, DSkernel, DSstride, DSpadding, dilation, groups = in_channels),

        nn.BatchNorm3d(in_channels),
        nn.ReLU(inplace=True),
        nn.Conv3d(in_channels, out_channels, 1 ,DSstride, DSpadding),
        nn.BatchNorm3d(out_channels),
        nn.ReLU(inplace=True)
    )

  def forward(self, input):
    return self.dsconv(input)


class DRDB(nn.Module):
      def __init__(self, in_channels, out_channels, DSkernel, DSstride, DSpadding):
        super(DRDB, self).__init__()
        
        #second parameter: in_channels or out_channels??
        self.conv_1 = DSconv(in_channels, in_channels, DSkernel, DSstride, DSpadding, dilation = 1)
        self.conv_2 = DSconv(in_channels, in_channels, DSkernel, DSstride, DSpadding, dilation = 2)
        self.conv_3 = DSconv(in_channels, in_channels, DSkernel, DSstride, DSpadding, dilation = 3)
        self.conv_4 = DSconv(in_channels, in_channels, DSkernel, DSstride, DSpadding, dilation = 4)
        #what does g0 mean?
        self.g0 = DSconv(in_channels, in_channels, 1, DSstride, DSpadding, dilation = 4)

      def forward(self, x):
        x1 = self.conv_1(x)
        concat_1 = torch.cat([x1,x], dim=1) # concatting the input and output of the same convolution
        x2 = self.conv_2(concat_1)
        concat_2 = torch.cat([x2,x11], dim=1)
        x3 = self.conv_3(concat_2)
        concat_3 = torch.cat([x3,x22], dim=1)
        x4 = self.conv_4(concat_3)
        concat_4 = torch.cat([x4,x33], dim=1)
        xg0 = self.g0(concat_4)
        
        return xg0 + x

        #Please write a full commit about your variable names
        # I've renamed some of them, others are left


Model

In [ ]:
class PLSnet(nn.Module):
  def __init__(self, in_channels, out_channels, DSkernel, DSstride, DSpadding):
    super(PLSnet, self).__init__()

    self.TLupsample = nn.Upsample(scale_factor=2, mode='trilinear')
    self.TLdownsample = nn.functional.interpolate(scale_factor= 1/2, mode='trilinear')
    self.oneconv = nn.Conv3d(in_channels, out_channels, 1 ,DSstride, DSpadding)
    self.softmaxAF = nn.Softmax()

  def forward(self, InputImage):
    # resolution = 1 in encoder
    g1 = DSconv(InputImage)
    InputImage = self.TLdownsample(InputImage)
    g1 = torch.cat([g1,InputImage], dim=1)
    g1 = DRDB(g1)
    # resolution = 2 in encoder
    g2 = DSconv(g1)
    InputImage = self.TLdownsample(InputImage)
    g2 = torch.cat([g2,InputImage], dim=1)
    g2 = DRDB(g2)
    g2 = DRDB(g2) # Don't know, but the paper's architecture has a DRDBx2! and I don't know what's that mean!
    # resolution = 3 in encoder
    g3 = DSconv(g2)

    InputImage = self.TLdownsample(InputImage)
    g3 = torch.cat([g3,InputImage], dim=1)
    g3 = DRDB(g3)
    g3 = DRDB(g3)
    g3 = DRDB(g3)
    g3 = DRDB(g3)
    g3 = DSconv(g3)
    g3d = self.TLupsample(g3) # g3 that got upsample and is placed in decoder
    # resolution = 2 in decoder
    g2d = torch.cat([g3d,g2], dim=1)
    g2d = DSconv(g2d)
    g2d = self.TLupsample(g2d)
    # resolution = 1 in decoder
    g1d = torch.cat([g2d,g1], dim=1)
    g1d = DSconv(g1d)
    g1d = self.TLupsample(g1d)
    # resolution = 0 in decoder
    g0d = self.oneconv(g1d)
    return self.Softmax(g0d) 

In [ ]:
optimizer = torch.optim.Adam(filter(
    lambda p : p.requires_grad, model.parameters()),
    lr = 0.001
)

In [ ]:
model = PLSnet()
model = model.cuda()

In [ ]:
from time import time

In [ ]:
iters_per_epoch = int(np.ceil(1.0 * len(x_train) / batch_size))

for e in range(epochs):
    t_start = time()

    model.train() # training phase

    # shuffling
    inds = np.arange(len(x_train))
    np.random.shuffle(inds)

    epoch_loss = 0
    true_positive = 0


    # iterating over the whole training set
    for iter in range(iters_per_epoch):

        batch_inds = inds[iter * batch_size: min(len(inds), (iter + 1) * batch_size)]

        # reshaping placeholders
        if len(batch_inds) != len(batch_x_placeholder):
            batch_x_placeholder.resize_([len(batch_inds), 1, 28, 28])
            batch_y_placeholder.resize_([len(batch_inds)])

        batch_x_placeholder.copy_(torch.Tensor(x_train[batch_inds, np.newaxis, :, :]))
        batch_y_placeholder.copy_(torch.Tensor(y_train[batch_inds].astype(int)))

        b_decision, b_loss = model(batch_x_placeholder, batch_y_placeholder)
        b_decision = b_decision.cpu().numpy()
      
        epoch_loss += float(b_loss) / iters_per_epoch  # CARE: WE SHOULD USE FLOAT OVER LOSS
        true_positive += np.sum(y_train[batch_inds].astype(int) == b_decision)

        b_loss.backward() # calculates derivations

        optimizer.step()
        optimizer.zero_grad() # CARE: MUST DO

    epoch_train_accuracy = true_positive * 100.0 / len(x_train)
    train_loss[e] = epoch_loss
    train_acc[e] = epoch_train_accuracy
    
 # Validating over validation data
    with torch.no_grad():
        model.eval()  # validation phase

        val_inds = np.arange(len(x_val))

        val_iters_per_epoch = int(np.ceil(1.0 * len(x_val) / batch_size))

        epoch_validation_loss = 0
        val_true_positive = 0


        # iterating over the whole training set
        for iter in range(val_iters_per_epoch):

            val_batch_inds = val_inds[iter * batch_size: min(len(val_inds), (iter + 1) * batch_size)]

            # reshaping placeholders
            if len(val_batch_inds) != len(batch_x_placeholder):
                batch_x_placeholder.resize_([len(val_batch_inds), 1, 28, 28])
                batch_y_placeholder.resize_([len(val_batch_inds)])

            batch_x_placeholder.copy_(torch.Tensor(x_val[val_batch_inds, np.newaxis, :, :]))
            batch_y_placeholder.copy_(torch.Tensor(y_val[val_batch_inds].astype(int)))

            b_decision, b_loss = model(batch_x_placeholder, batch_y_placeholder)
            b_decision = b_decision.cpu().numpy()
        
            epoch_validation_loss += float(b_loss) / val_iters_per_epoch  # CARE: WE SHOULD USE FLOAT OVER LOSS
            val_true_positive += np.sum(y_val[val_batch_inds].astype(int) == b_decision)
                
        epoch_validation_accuracy = val_true_positive * 100.0 / len(x_val)
        val_loss[e] = epoch_validation_loss
        val_acc[e] = epoch_validation_accuracy
        # TO Complete
    
    print(f'Train epoch Loss: {epoch_loss:.4f}, train accuracy: {epoch_train_accuracy:.2f}, Validation Loss: {epoch_validation_loss:.4f}, validation accuracy: {epoch_validation_accuracy:.2f}')

    # Saving the model and optimizer state
    torch.save({
            'epoch': e,
            'optimizer_state_dict': optimizer.state_dict(),
            'model_state_dict': model.state_dict(),
            'train_loss': epoch_loss,
            'train_accuracy': epoch_train_accuracy,
            'validation_loss': epoch_validation_loss,
            'validation_accuracy': epoch_validation_accuracy
        }, 'epoch_%d_state.pt' % e)

    print('Epoch %d ended in %.2f secs.' % (e, time() - t_start,))
